# 한국어를 전처리하기

한국에 사는 이상, 영어만 분석할 수는 없는 노릇일 것이다(...)

## KoNLPy 한국어 처리 패키지

In [1]:
import konlpy
konlpy.__version__

'0.5.2'

In [2]:
from konlpy.corpus import kolaw
kolaw.fileids()

['constitution.txt']

In [3]:
kolaw.open('constitution.txt').read()[:100]

'대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의'

## 형태소 분석
한국어의 형태소 분석 방법, KoNLPY에서는 다양한 형태소 분석을 제공하고 있다.

- nouns : 명사를 추출한다.
- morphs : 형태소 추출
- pos : 품사를 부착한다.

In [4]:
from konlpy.tag import *

In [5]:
text = '아름다운 이 땅에 금수강산에 단군 할아버지가 터 잡으시다'

형태소 분석기에는 여러 종류가 있다.
- hannanum
- kkma
- komoran
- mecab -> 이유는 모르겠는데, 안된다... 윈도우즈에서 안되는 것으로 추정
- okt

In [6]:
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [7]:
# KoNLPy’s Mecab() class is not supported on Windows machines.
# mecab = Mecab()

### 명사 추출

In [8]:
print(hannanum.nouns(text))

['땅', '금수강산', '단군', '할아버지']


In [9]:
print(kkma.nouns(text))

['땅', '금수강산', '단군', '할아버지']


In [10]:
print(komoran.nouns(text))

['땅', '금수강산', '단군', '할아버지', '터']


In [11]:
print(okt.nouns(text))

['이', '땅', '금수강산', '단군', '할아버지', '터']


### 형태소 추출

In [12]:
print(hannanum.morphs(text))

['아름답', '은', '이', '땅', '에', '금수강산', '에', '단군', '할아버지', '가', '트', '어', '잡', '으시다']


In [13]:
print(kkma.morphs(text))

['아름답', 'ㄴ', '이', '땅', '에', '금수강산', '에', '단군', '할아버지', '가', '트', '어', '잡', '으시', '다']


In [14]:
print(komoran.morphs(text))

['아름답', 'ㄴ', '이', '땅', '에', '금수강산', '에', '단군', '할아버지', '가', '터', '잡', '으시', '다']


In [15]:
print(okt.morphs(text))
print(okt.morphs(text, stem = True)) # 표제어 단위로 추출되는 기능 보유

['아름다운', '이', '땅', '에', '금수강산', '에', '단군', '할아버지', '가', '터', '잡으시다']
['아름답다', '이', '땅', '에', '금수강산', '에', '단군', '할아버지', '가', '터', '잡다']


### 품사 부착
때론, 품사에 대해 알아야 하는 경우가 있을 것이다. 각 형태소 추출기별로, 별도의 다른 기호를 사용한다...

In [16]:
print(hannanum.pos(text))

[('아름답', 'P'), ('은', 'E'), ('이', 'M'), ('땅', 'N'), ('에', 'J'), ('금수강산', 'N'), ('에', 'J'), ('단군', 'N'), ('할아버지', 'N'), ('가', 'J'), ('트', 'P'), ('어', 'E'), ('잡', 'P'), ('으시다', 'E')]


In [17]:
print(kkma.pos(text))

[('아름답', 'VA'), ('ㄴ', 'ETD'), ('이', 'MDT'), ('땅', 'NNG'), ('에', 'JKM'), ('금수강산', 'NNG'), ('에', 'JKM'), ('단군', 'NNG'), ('할아버지', 'NNG'), ('가', 'JKS'), ('트', 'VV'), ('어', 'ECS'), ('잡', 'VV'), ('으시', 'EPH'), ('다', 'EFN')]


In [18]:
print(komoran.morphs(text))

['아름답', 'ㄴ', '이', '땅', '에', '금수강산', '에', '단군', '할아버지', '가', '터', '잡', '으시', '다']


In [19]:
print(okt.morphs(text))

['아름다운', '이', '땅', '에', '금수강산', '에', '단군', '할아버지', '가', '터', '잡으시다']


### 어절 추출

In [20]:
# okt에ㄷ만 존재하는 것으로 보인다.
print(okt.phrases(text))

['아름다운 이', '아름다운 이 땅', '금수강산', '단군', '단군 할아버지', '할아버지']


## 정규화 및 불용어 제거

In [21]:
import urllib.request
import pandas as pd
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

In [22]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
data = pd.read_table('ratings.txt') # 데이터프레임에 저장
data[:10]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


### 정규표현식을 통한 데이터 정제

In [23]:
data['document'] = data.document.apply(lambda s : str(s))

In [24]:
# 띄어쓰기와 한글로 된 것들을 "제외하고" 모두 토큰화를 한다.
import re
import string
def clean_text(text):
    text = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",text)
    return text

In [25]:
data['document'] = data.document.apply(lambda s : clean_text(s))

In [26]:
data

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ,0
199996,3302770,쓰레기,0
199997,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


### 토큰화 및 불용어 제거
불용어를 제거하는 방법 중 여러가지가 있으나, 여기서는 직접 불용어 집합을 정의한다.  
토큰화의 경우 형태소 분석기를 사용하며, 여기서는 okt를 사용한다.

In [27]:
tokenizer = Okt()
def text_preprocess(text):
    stopwords = ['은', '는', '이', '가', '의', '을', '를', 
                 '도', '한', '하다', '와', '과', '에', '게',
                '그', '저', '들', '다']
    temp = tokenizer.morphs(text)
    temp = [word for word in temp if not word in stopwords]
    return temp

In [28]:
data['document'] = data.document.apply(lambda s : text_preprocess(s))

In [29]:
data

,id,document,label
0,8112052,"[어릴, 때, 보고, 지금, 다시, 봐도, 재밌어요, ㅋㅋ]",1
1,8132799,"[디자인, 배우는, 학생, 으로, 외국, 디자이너, 일군, 전통, 통해, 발전, 해...",1
2,4655635,"[폴리스스토리, 시리즈, 부터, 뉴, 까지, 버릴께, 하나, 없음, 최고]",1
3,9251303,"[연기, 진짜, 개, 쩔구나, 지루할거라고, 생각, 했는데, 몰입, 해서, 봤다, ...",1
4,10067386,"[안개, 자욱한, 밤하늘, 떠, 있는, 초승달, 같은, 영화]",1
...,...,...,...
199995,8963373,"[포켓, 몬스터, 짜가, ㅡㅡ]",0
199996,3302770,[쓰레기],0
199997,5458175,"[완전, 사이코, 영화, 마지막, 더욱더, 영화, 질, 떨어, 트, 린다]",0
199998,6908648,"[왜, 난, 재미없었지, ㅠㅠ, 라따뚜이, 보고나서, 스머프, 봐서, 런가, ㅋㅋ]",0


In [30]:
vocab = FreqDist(np.hstack(data.document.values))

In [31]:
len(vocab)

117276

토큰화 시킨 후, 리뷰 안에 들어있는 단어 개수는 총 117280개이다. 이 중 상위 빈도수에 대해서만 추출하고 싶다.

In [32]:
vocab_size = 1000
vocab = vocab.most_common(vocab_size)

이렇게 단어집합을 1000개로 압축 가능하다.

## 각 단어에 고유 정수 부여하기(word2index)

In [33]:
word_to_index = {word[0] : index + 2 for index, word in enumerate(vocab)}
word_to_index['pad'] = 1
word_to_index['unk'] = 0

In [36]:
encoded = []
for tokens in data.document.values:
    temp = []
    for word in tokens:
        try:
            temp.append(word_to_index[word])
        except KeyError:
            temp.append(word_to_index['unk']) # 없는 단어
    encoded.append(temp)

In [39]:
encoded[:3]

[[436, 36, 30, 88, 63, 92, 318, 37],
 [0,
  0,
  0,
  8,
  0,
  0,
  0,
  0,
  989,
  0,
  0,
  0,
  0,
  0,
  372,
  355,
  726,
  0,
  547,
  97,
  48,
  0,
  0,
  0,
  70,
  0,
  698,
  70,
  29,
  583,
  0,
  0,
  0,
  55,
  0,
  14,
  0],
 [0, 218, 123, 0, 48, 0, 65, 254, 15]]

다음과 같이 길이가 모두 다름을 확인해 볼 수 있다. 이 길이를 동일하게 맞춰주는 작업을 padding이라고 한다.

## Padding

In [40]:
max_len = max([len(s) for s in encoded])
for labeled in encoded:
    if len(labeled) < max_len:
        labeled += [word_to_index['pad']] * (max_len - len(labeled))

In [43]:
encoded[:3]

[[436,
  36,
  30,
  88,
  63,
  92,
  318,
  37,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [0,
  0,
  0,
  8,
  0,
  0,
  0,
  0,
  989,
  0,
  0,
  0,
  0,
  0,
  372,
  355,
  726,
  0,
  547,
  97,
  48,
  0,
  0,
  0,
  70,
  0,
  698,
  70,
  29,
  583,
  0,
  0,
  0,
  55,
  0,
  14,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [0,
  218,
  123,
  0,
  48,
  0,
  65,
  254,
  15,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

이러면, 다음과 같이 padding이 되었음을 확인해 볼 수 있다.